# Conclusions about previous versions

After performing lond research of the topic was discovered that PointNet approach proves to be a good one referring to the results of performing classification of the emotions. Other approach require much deeper NNs to make them efficient and this one allows reaching great performance. But, some moments should be mentioned:

1. In terms of performing classification of emotions, best model achieves result of 63% accuracy, which is great considering amount of emotions. Presented model performance deviate between 60-62% of the accuracy, which is comparable to the best results. Still, it is required to validate model over even bigger sets to make sure that accuracy scores are correct.
2. Chosen approach gives one great advantage and one great disadvantage. Advantage is that model uses Google's MediaPipe FaceMesh model to get 3D scan of the face, meaning that model abstracts as far as possible from image noise, skin color of the person, conditions of light in the photo and so on. It takes only facial landmarks (in other words, shape of face elements) and based on them finds emotion type. Disadvantage is in limitations caused by chosen approach. It highly depends on the 3D scan model both performance and efficiency. Another moment is that because of scans there is no efficient way to generate new data out of the present one. While images can be modified to abstract model from exact shapes and colors (or other aspects), there is no way to modify scans, because it will cause shape changes and each is able to change entirely face interpretation.
3. Regression over ```valence``` and ```arousal``` has proven to be inefficient because in the best case author was able to hit 0.45 R2 score for both of the variables. Because of such low scores it is questionable either if there are problems with how variables were calculated for the dataset, or about efficiency of the chosem approach in terms of regression

Here are final steps with forming classification model that will be used for the 100% project protection and that will be loaded into the final application

# Loading all libs and data

##  Loading libs

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os.path
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from tensorflow.keras import layers

import gc

from point_net_api import *

tf.random.set_seed(1234)

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

import sklearn.metrics as sm

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17302335861178982308
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5750390784
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17262008492494526157
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]
1 Physical GPUs, 1 Logical GPUs
Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 3070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6



## Loading data and dropping unnecessary fields

In [2]:
train_df = read_all_pickles(package_name='pkls_valence_arousal', file_specification="_landmarks_valence_arousal.pkl")
print('original length = ' + str(len(train_df)))

#   drop empty records
train_df.dropna(inplace=True)

#   ignoring the last three labels because they are not providing necessary information
train_df = train_df[(train_df.emotion != 8) & (train_df.emotion != 9) & (train_df.emotion != 10)]
train_df.drop(columns=['valence', 'arousal'], inplace=True)

print('updated length = ' + str(len(train_df)))
train_df.head(10)

original length = 409797
updated length = 272340


,path,emotion,landmarks
400002,Manually_Annotated_Images/325/f489e8169ea30897...,1,"[[0.5394599437713623, 0.676323413848877, -0.08..."
400003,Manually_Annotated_Images/264/224ee36d52b497b1...,0,"[[0.5218021273612976, 0.7440579533576965, -0.1..."
400005,Manually_Annotated_Images/980/8262220f6a3bcfd7...,1,"[[0.5117629766464233, 0.7039216160774231, -0.0..."
400007,Manually_Annotated_Images/609/5b3d3b8b9d89deaf...,1,"[[0.47549158334732056, 0.7176483273506165, -0...."
400009,Manually_Annotated_Images/958/7b235cdca2ca0430...,2,"[[0.5392422080039978, 0.7580064535140991, -0.0..."
400013,Manually_Annotated_Images/715/bff238e798b70ccf...,6,"[[0.49899062514305115, 0.7014129161834717, -0...."
400014,Manually_Annotated_Images/514/f92fad99d07e4b9e...,4,"[[0.47227370738983154, 0.6721517443656921, -0...."
400015,Manually_Annotated_Images/728/9629f4764d5956d9...,1,"[[0.519411563873291, 0.739055871963501, -0.060..."
400016,Manually_Annotated_Images/866/ec55abeb8a12d104...,1,"[[0.5619162917137146, 0.6456629633903503, -0.0..."
400017,Manually_Annotated_Images/907/7afaf57f140a6270...,1,"[[0.5264552235603333, 0.7333130240440369, -0.0..."


In [3]:
X = np.array(list(map(np.float64, np.array(train_df['landmarks']))))
Y = np.array(train_df['emotion'])

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

del X
del Y
del train_df

gc.collect()

22

In [4]:
train_tf_df = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_tf_df = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_tf_df = train_tf_df.shuffle(len(x_train)).batch(32)
test_tf_df = test_tf_df.shuffle(len(x_train)).batch(32)

In [5]:
del y_train
del y_test
gc.collect()

del x_train
del x_test
gc.collect()

0

# VERSION 3.0 - base model

## Make NN

In [6]:
first_degree, second_degree, third_degree, fourth_degree = 128, 256, 512, 1024

inputs = keras.Input(shape=(468, 3))

x = tnet(inputs, 3, first_degree, second_degree, third_degree, fourth_degree, 'leaky_relu')
x = conv_bn(x, 64, activation_function='leaky_relu', kernel_size=3)
x = conv_bn(x, 64, activation_function='leaky_relu', kernel_size=3)
x = tnet(x, 64, first_degree, second_degree, third_degree, fourth_degree, activation_function='leaky_relu')
x = conv_bn(x, 64, activation_function='leaky_relu', kernel_size=3)
x = conv_bn(x, first_degree, activation_function='leaky_relu', kernel_size=3)
x = conv_bn(x, third_degree, activation_function='leaky_relu', kernel_size=3)
x = layers.GlobalMaxPooling1D()(x)
x = dense_bn(x, second_degree, activation_function='leaky_relu')
x = layers.Dropout(0.25)(x)
x = dense_bn(x, first_degree, activation_function='leaky_relu')
x = layers.Dropout(0.25)(x)

outputs = layers.Dense(8, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="pointnet_class")
model.summary()

Model: "pointnet_class"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 468, 3)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 468, 32)      128         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 468, 32)     128         ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 leaky_re_lu (LeakyReLU)        (None, 468, 32)      0           ['batch_normalizatio

## Train

In [7]:
model.compile(loss="sparse_categorical_crossentropy",
            optimizer=keras.optimizers.Adam(learning_rate=0.001),
            metrics=["sparse_categorical_accuracy"],)

model.fit(train_tf_df, epochs=7, validation_data=test_tf_df)

Epoch 1/7
6809/6809 [==============================] - 422s 60ms/step - loss: 3.3311 - sparse_categorical_accuracy: 0.5988 - val_loss: 3.2649 - val_sparse_categorical_accuracy: 0.6163
Epoch 2/7
6809/6809 [==============================] - 410s 60ms/step - loss: 3.1957 - sparse_categorical_accuracy: 0.6256 - val_loss: 3.1246 - val_sparse_categorical_accuracy: 0.6424
Epoch 3/7
6809/6809 [==============================] - 412s 60ms/step - loss: 3.1375 - sparse_categorical_accuracy: 0.6391 - val_loss: 3.1268 - val_sparse_categorical_accuracy: 0.6402
Epoch 4/7
6809/6809 [==============================] - 411s 60ms/step - loss: 3.1196 - sparse_categorical_accuracy: 0.6446 - val_loss: 3.1977 - val_sparse_categorical_accuracy: 0.6183
Epoch 5/7
6809/6809 [==============================] - 412s 60ms/step - loss: 3.1008 - sparse_categorical_accuracy: 0.6496 - val_loss: 3.0704 - val_sparse_categorical_accuracy: 0.6568
Epoch 6/7
6809/6809 [==============================] - 414s 61ms/step - loss: 3.

## Validate and save model

In [10]:
model.save_weights('models/v3_point_net/')

In [9]:
index, matches = 0, 0

for batch in test_tf_df.as_numpy_iterator():
    batch_predictions = model.predict(batch[0])
    batch_suggestions = [np.argmax(prediction) for prediction in batch_predictions]
    for inner_index in range(len(batch_suggestions)):
        suggestion = batch_suggestions[inner_index]
        if suggestion == batch[1][inner_index]:
            matches += 1
    index += 1
    
print(matches / (index * 32))

1/1 [==============================] - 1s 750ms/step
0.6551306517909571


# Version 3.1 - deeper model, higher learning rate, bigger conv kernel

## Make NN

In [6]:
first_degree, second_degree, third_degree, fourth_degree = 256, 512, 1024, 2048

inputs = keras.Input(shape=(468, 3))

x = tnet(inputs, 3, first_degree, second_degree, third_degree, fourth_degree, 'leaky_relu')
x = conv_bn(x, 64, activation_function='leaky_relu', kernel_size=3)
x = conv_bn(x, 64, activation_function='leaky_relu', kernel_size=3)
x = tnet(x, 64, first_degree, second_degree, third_degree, fourth_degree, activation_function='leaky_relu')
x = conv_bn(x, 64, activation_function='leaky_relu', kernel_size=3)
x = conv_bn(x, first_degree, activation_function='leaky_relu', kernel_size=3)
x = conv_bn(x, third_degree, activation_function='leaky_relu', kernel_size=3)
x = layers.GlobalMaxPooling1D()(x)
x = dense_bn(x, second_degree, activation_function='leaky_relu')
x = layers.Dropout(0.25)(x)
x = dense_bn(x, first_degree, activation_function='leaky_relu')
x = layers.Dropout(0.25)(x)

outputs = layers.Dense(8, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="pointnet_class")
model.summary()

Model: "pointnet_class"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 468, 3)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 468, 32)      128         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 468, 32)     128         ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 leaky_re_lu (LeakyReLU)        (None, 468, 32)      0           ['batch_normalizatio

## Training

In [7]:
model.compile(loss="sparse_categorical_crossentropy",
            optimizer=keras.optimizers.Adam(learning_rate=0.001),
            metrics=["sparse_categorical_accuracy"],)

model.fit(train_tf_df, epochs=10, validation_data=test_tf_df)

Epoch 1/10
6809/6809 [==============================] - 902s 130ms/step - loss: 3.3956 - sparse_categorical_accuracy: 0.5894 - val_loss: 3.2645 - val_sparse_categorical_accuracy: 0.6137
Epoch 2/10
6809/6809 [==============================] - 865s 127ms/step - loss: 3.2414 - sparse_categorical_accuracy: 0.6164 - val_loss: 3.1969 - val_sparse_categorical_accuracy: 0.6303
Epoch 3/10
6809/6809 [==============================] - 869s 127ms/step - loss: 3.1757 - sparse_categorical_accuracy: 0.6303 - val_loss: 3.4469 - val_sparse_categorical_accuracy: 0.6083
Epoch 4/10
6809/6809 [==============================] - 869s 127ms/step - loss: 3.1298 - sparse_categorical_accuracy: 0.6415 - val_loss: 3.1513 - val_sparse_categorical_accuracy: 0.6326
Epoch 5/10
6809/6809 [==============================] - 869s 127ms/step - loss: 3.0993 - sparse_categorical_accuracy: 0.6505 - val_loss: 3.0771 - val_sparse_categorical_accuracy: 0.6604
Epoch 6/10
6809/6809 [==============================] - 869s 128ms/ste

## Validating model and saving

In [8]:
index, matches = 0, 0

for batch in test_tf_df.as_numpy_iterator():
    batch_predictions = model.predict(batch[0])
    batch_suggestions = [np.argmax(prediction) for prediction in batch_predictions]
    for inner_index in range(len(batch_suggestions)):
        suggestion = batch_suggestions[inner_index]
        if suggestion == batch[1][inner_index]:
            matches += 1
    index += 1
    
print(matches / (index * 32))

1/1 [==============================] - 1s 756ms/step
0.6497908103347034


In [9]:
model.save_weights('models/v3_1_point_net/')